The idea is to get the genes(the genes that were translated, not intergenes) in the vicinity of the genes in the "predicted_gene_blocks.fa" file.

![img](geneblocks.png)

For example, in the above case, the red block is a gene block in our context and we have a modifier, an immunity and a trasnport gene there. We are taking all the genes that are in between them and also around them for a certain radius.



In [ ]:
"""
Make the list of lists, where the members are the start and end number of 
gene blocks in the file predicted_gene_blocks.fa
"""

import os
os.chdir('/home/nafizh/try_Iddo_approaches/')

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os.path

handle = open("predicted_gene_blocks.fa", "rU")
mydict = dict()
myset = set()

# taking out accession number, and their start and end,
# and putting it into a dictionary where key is the 
# accession number and value is a list of lists with start,
# and end numbers
for record in SeqIO.parse(handle, "fasta") :
    temp = record.id.split('|')
    #print temp[0][10:20]
    myset.add(temp[0][10:18])
    if temp[0][10:18] in mydict:
        mydict[temp[0][10:18]] = mydict[temp[0][10:18]] + [temp[2][6:], temp[3][4:]]
    else:
        mydict[temp[0][10:18]] = [temp[2][6:], temp[3][4:]]



# so that the lists within the list are sorted
# need them sorted because they are start and end of 
# protein sequence 
for key in mydict:
    mydict[key].sort()

print mydict['AE001437']
print mydict['AE016877']

In [ ]:
"""
This code should get all the adjacent and in between genes of the genes 
predicted in the "predicted_gene_blocks.fa" file
"""

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os.path

os.chdir('/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction')

def get_genes(input_file, output_file, my_key):
    try:
        """get the desired gene blocks from .gbk files"""
        mysequences = []
        threshold = 1000
        handle = open(input_file)

        # the start position of the gene with lowest starting position
        predicted_gene_block_start = mydict[my_key][0]
        # the end position of the gene with highest ending position
        predicted_gene_block_end = mydict[my_key][-1]


        lower_thres_hold = int(predicted_gene_block_start) - threshold
        upper_thres_hold = int(predicted_gene_block_end) + threshold

        strand_dict = {-1: '-', 1 : '+'}

        seq_record = SeqIO.parse(open(input_file), "genbank").next()
        for feature in seq_record.features:
            if feature.type == 'CDS':
                mystart = feature.location.start.position
                myend = feature.location.end.position
                #print mystart
                #print myend
                if lower_thres_hold <= mystart and myend <= upper_thres_hold and str(mystart + 1) not in mydict[my_key] and str(myend) not in mydict[my_key]:
                    #print "hello"
                    gene_seq = seq_record.seq[mystart : myend]
                    mysequences.append(
                          SeqRecord(gene_seq, id="%s:%d-%d%s" % (seq_record.name,mystart + 1, myend, strand_dict[feature.strand]),
                          description="%s %d-%d %s" % (seq_record.name, mystart + 1, myend, strand_dict[feature.strand])))


        SeqIO.write(mysequences, open(output_file, 'w'), "fasta")
    except Exception as e:
        print "Error at", input_file, e

#get_genes('AE001437.gbk', 'rahi.fa', 'AE001437')

root_dir = "/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction"

"""Going through all .gbk files and getting desired gene blocks from them"""
for root, subFolders, files in os.walk(root_dir):
    for fname in files:
        genome_files = []
        ext = os.path.splitext(os.path.basename(fname))[1]
        name = os.path.splitext(os.path.basename(fname))[0]
        if ext == ".gbk":
            output_file = name + "_desired_genes.fa"
            absfile= os.path.join(root,fname)
            my_key = fname.split('.')[0]
            get_genes(absfile, output_file, my_key)
            
print "Calculation done"

**After we get the \*_desired_genes.fa files, we cat them all to a file called all_desired_genes.fa and then we  blast clust all the sequences on that file. We then translated all the sequences with transeq by the clusters we got from doing blast clust. We cat all those translations to a file called all_translations.fa and then blasted that file against the nr database. Finally, we got the blast result in the "blast_res_for_all_translations.xml" file. We then parsed it by the following code.**

In [ ]:
# Getting only proteins that have some unknown proteins annotated to them
# for blast result of all 777 protein sequences

import os
os.chdir("/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction/desired_gene_blocks/translations")
from Bio.Blast import NCBIXML

result_handle = open("blast_res_for_all_translations.xml")
res_file = open("new_result_all_translations.fasta", 'w')
sec_file = open("new_result_all_translations1.fasta", 'w')

blast_records = NCBIXML.parse(result_handle)

track_l_all = []

for i, blast_record in enumerate(blast_records):
    for j, alignment in enumerate(blast_record.alignments): # j is basically the hit number within a blast record/blast_iteration
        if j == 1: # only the top hit
            break
        for hsp in alignment.hsps:
            if "unknown function" in alignment.title:
                if i + 1 in track_l_all: # so that does not enter the same record/blast_iteration again
                    continue
                else:
                    res_file.write(">seq num: %d | hit num: %d | query: %s | hit-id: %s | hit-length: %d | hit-score: %d | e-value: %s|\n\n" % (i + 1, j + 1, blast_record.query, alignment.title, alignment.length, hsp.score, (hsp.expect)))
                    res_file.write(hsp.query[0:90] + "...\n")
                    res_file.write(hsp.match[0:90] + "...\n")
                    res_file.write(hsp.sbjct[0:90] + "...\n")
                    res_file.write("\n\n")
                    sec_file.write(">seq num: %d | hit num: %d | query: %s | hit-id: %s | hit-length: %d | hit-score: %d | e-value: %s|\n\n\n" % (i+1, j + 1, blast_record.query, alignment.hit_id, alignment.length, hsp.score, (hsp.expect)))
                    track_l_all = track_l_all + [i + 1]
        

            
res_file.close()
sec_file.close()
print "Calculation done"   

**We have got all the bacteriocin hits with "unknown function". Now all the following code is to annotate them nicely with protein and dna sequences with their accession ids and species name.**

In [ ]:
# Creating a temporary file of bacteriocins with unknown protein hits in a
# format similar to all_translations.fa

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os
os.chdir("/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction/desired_gene_blocks/translations")

bac_file = "all_bacteriocins_with_unknown_protein_hit.fa"
cd_hit_handle = open("all_translations.fa")

mysequences = []

j = 0

for i, record in enumerate(SeqIO.parse(cd_hit_handle, "fasta")):
    if i + 1 in track_l_all: # track_l_all comes from the previous cell
        mysequences.append(SeqRecord(record.seq, id= ("seq num: %d | serial num: %d |") % (i + 1, j + 1), 
                                                         description= record.description))
        j += 1
    SeqIO.write(mysequences, open(bac_file, 'w'), "fasta")

print "Done"

In [ ]:
'''
Creating a file with the desired putative bacteriocins with accession id, species name and co-ordinate number
'''

# Getting the accession ids as value in a dictionary where the key are the cordinates

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os
os.chdir("/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction/desired_gene_blocks/translations")

bac_file = open("all_bacteriocins_with_unknown_protein_hit.fa")
all_desired_gene = open("all_desired_genes.fa")

coordinates = []

for i, record in enumerate(SeqIO.parse(bac_file, "fasta")):
    coordinates.append(record.description.split("|")[2].split(" ")[2])

accessions = dict()
for i, record in enumerate(SeqIO.parse(all_desired_gene, "fasta")):
    #print record.description.split(":")[0]
    for coord in coordinates:
        if coord in record.description:
            temp = record.description.split(":")[0]
            accessions[coord] = temp

print accessions
            
print len(accessions)

In [ ]:
# Getting the genome ids for the accession ids

from Bio import Entrez
Entrez.email     = "nafizh@iastate.edu"

accessions_list = []

for value in accessions.itervalues():
    accessions_list.append(value) 

print accessions_list
print len(accessions_list)

genomeAccessions1 = accessions_list[0:20]
search1           = " ".join(genomeAccessions1)
print search1
handle1           = Entrez.read(Entrez.esearch(db="nucleotide", term=search1, retmode="xml"))
genomeIds1        = handle1['IdList']
print genomeIds1
print len(genomeIds1)

# genomeAccessions2 = accessions_list[20:40]
# search2           = " ".join(genomeAccessions2)
# print search2
# handle2           = Entrez.read(Entrez.esearch(db="nucleotide", term=search2, retmode="xml"))
# genomeIds2        = handle2['IdList']
# print genomeIds2
# print len(genomeIds2)
# # # handle = Entrez.efetch(db="nucleotide", id="186972394", retmode="xml")
# # # record = Entrez.read(handle)
# # # handle.close()
# # # record[0]["GBSeq_definition"]

# genomeAccessions3 = accessions_list[40:]
# search3           = " ".join(genomeAccessions3)
# print search3
# handle3           = Entrez.read(Entrez.esearch(db="nucleotide", term=search3, retmode="xml"))
# genomeIds3        = handle3['IdList']
# print genomeIds3
# print len(genomeIds3)

In [ ]:
# Creating a dicitionary where the key is the coordinate number and the values are the
# respective genome ids and accession ids

genome_ids = genomeIds1 
#+ genomeIds2 + genomeIds3

print genome_ids
accession_dict = dict()

print accessions

track = 0
for key, val in accessions.iteritems():
    temp = [] + [val]
    #accession_dict[key] = temp.append(genome_ids[track])
    #print temp
    temp1 = temp + [(str(genome_ids[track]))]
    #print temp1
    accession_dict[key] = temp1
    track += 1
    
print accession_dict
print len(accession_dict)

In [ ]:
# Getting species name from the genome Ids

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os
os.chdir("/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction/desired_gene_blocks/translations")

all_handle = open("all_translations.fa")

# dictionary where keys are coordinates and values are respective species names
species_name = dict()

for i, record in enumerate(SeqIO.parse(all_handle, "fasta")):
    if i + 1 in track_l_all:
        print record.description
        my_key = record.description.split(" ")[1]
        my_id = accession_dict[my_key][1]
        handle = Entrez.efetch(db="nucleotide", id = my_id, retmode="xml")
        my_record = Entrez.read(handle)
        species_name[my_key] = my_record[0]["GBSeq_definition"]
        handle.close()
        
print species_name
print len(species_name)

In [ ]:
'''
Writing the file of putative bacteriocins with protein sequences for Iddo
output: "10_bacteriocins_with_unknown_protein_hit_with_ids.fa" file
'''

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os
os.chdir("/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction/desired_gene_blocks/translations")

bac_file = "10_bacteriocins_with_unknown_protein_hit_with_ids.fa"
all_handle = open("all_translations.fa")

mysequences = []

j = 0
count = 0

for i, record in enumerate(SeqIO.parse(all_handle, "fasta")):
    if i + 1 in track_l_all:
        print record.description
        my_key = record.description.split(" ")[1]
        mysequences.append(SeqRecord(record.seq, id= ("seq num: %d | serial num: %d | acession id: %s | species: %s|") % (i + 1, j + 1, accession_dict[my_key][0],
                             species_name[my_key]), description= record.description))
        j += 1
    SeqIO.write(mysequences, open(bac_file, 'w'), "fasta")

print "Done"

In [ ]:
'''
Writing the file of putative bacteriocins with dna sequences for Iddo
output : "10_bacteriocins_with_unknown_protein_hit_with_ids_dna_seq.fa" file
'''

from Bio import SeqIO, SeqFeature
from Bio.SeqRecord import SeqRecord
import os
os.chdir("/home/nafizh/try_Iddo_approaches/all_gbk_files_in_prediction/desired_gene_blocks/translations")

bac_file = "10_bacteriocins_with_unknown_protein_hit_with_ids_dna_seq.fa"
all_handle = open("all_translations.fa")
all_desired_gene = open("all_desired_genes.fa")

mysequences = []

j = 0
count = 0

# dictionary where keys are coordinates and values are respective sequences
seq_dict = dict()

for gene_record in SeqIO.parse(all_desired_gene, "fasta"):
    for key in accession_dict:
        if key in gene_record.description:
            seq_dict[key] = gene_record.seq

print seq_dict
print len(seq_dict)

for i, record in enumerate(SeqIO.parse(all_handle, "fasta")):
    if i + 1 in track_l_all:
        print record.description
        my_key = record.description.split(" ")[1]
        mysequences.append(SeqRecord(seq_dict[my_key], id= ("seq num: %d | serial num: %d | acession id: %s | species: %s|") % (i + 1, j + 1, accession_dict[my_key][0],
                             species_name[my_key]), description= record.description))
        j += 1
    SeqIO.write(mysequences, open(bac_file, 'w'), "fasta")

print "Done"